# Debug Dedicado: Context Manager

Este notebook foi criado para isolar e debugar falhas no `ContextManager`, focando em:
1. **Conectividade**: Validar se conseguimos listar tabelas e colunas via Spark.
2. **Segurança/Permissão**: Verificar se métodos como `.toPandas()` funcionam sem erros de bloqueio (WhiteList/Socket).
3. **Formatação de Prompt**: Garantir que a string de contexto gerada esteja correta e legível para o LLM.

In [0]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pyspark.sql import SparkSession

# Adiciona o diretório atual ao path para importação dos módulos
sys.path.append(os.getcwd())

## 1. Inicialização da Sessão Spark (Segura)
Tentamos pegar a sessão ativa para evitar conflitos.

In [0]:
try:
    spark = SparkSession.getActiveSession()
    if not spark:
        print("Criando nova sessão Spark (Ambiente Local)...")
        spark = SparkSession.builder.appName("DebugContextManager").getOrCreate()
    print(f"[OK] Spark Session: {spark}")
except Exception as e:
    print(f"[ERRO CRÍTICO] Não foi possível obter a SparkSession: {e}")

## 2. Teste Direto do ContextManager
Instanciamos a classe e chamamos o método principal. Se isso falhar, o erro está dentro da função.

In [0]:
from app.context_manager import ContextManager

try:
    print("Inicializando ContextManager...")
    context_mgr = ContextManager(spark)
    
    role = "logistics"
    print(f"Gerando schema para o papel: '{role}' (esperado: olist_sales e olist_logistics)...")
    
    context_output = context_mgr.get_schema_context(role)
    
    if context_output:
        print("\n[SUCESSO] Contexto Gerado com Sucesso!")
        print("="*40)
        print(context_output[:1000]) # Imprime os primeiros 1000 caracteres
        print("...(truncado)")
        print("="*40)
    else:
        print("\n[AVISO] O contexto retornou vazio. As queries funcionaram mas não encontraram tabelas/colunas.")

except Exception as e:
    print(f"\n[FALHA] Erro ao executar ContextManager: {e}")

## 3. Debug Granular (Simulação Manual)
Se o passo 2 falhou, executamos aqui **linha a linha** o que o código faz internamente.
Isso nos diz exatamente onde está o problema: na listagem de tabelas, na conversão pandas, ou no describe.

In [0]:
domains = ["olist_dataset.olist_sales", "olist_dataset.olist_logistics"]

print("Iniciando Debug Granular Manual...\n")

for domain in domains:
    print(f"--- Analisando Domínio: {domain} ---")
    
    # Passo A: Listar Tabelas
    try:
        print(f"1. Executando: SHOW TABLES IN {domain}")
        df_tables = spark.sql(f"SHOW TABLES IN {domain}").limit(20) # Limitando como no código
        
        print("2. Convertendo tabelas para Pandas...")
        pdf_tables = df_tables.toPandas()
        print(f"   > Sucesso! Encontradas {len(pdf_tables)} tabelas.")
        
        # Passo B: Iterar e Descrever
        for index, row in pdf_tables.iterrows():
            t_name = row['tableName']
            full_table = f"{domain}.{t_name}"
            print(f"   > Inspecionando tabela: {full_table}")
            
            try:
                print(f"     - Executando: DESCRIBE {full_table}")
                df_cols = spark.sql(f"DESCRIBE {full_table}")
                
                print("     - Convertendo colunas para Pandas...")
                pdf_cols = df_cols.toPandas()
                print(f"     - OK! {len(pdf_cols)} colunas recuperadas.")
                
                # Validação de formato (apenas amostra)
                first_col = pdf_cols.iloc[0]
                print(f"     - Exemplo de coluna: {first_col['col_name']} ({first_col['data_type']})")
                
            except Exception as e_desc:
                print(f"     [ERRO] Falha ao descrever tabela {full_table}: {e_desc}")
                
    except Exception as e_table:
        print(f"[ERRO] Falha crítica no domínio {domain}: {e_table}")
    
    print("\n")